In [1]:
"""
UTF-8, Python 3

------------------
Flaring SPI
------------------

Ekaterina Ilin, 2021, MIT License

De-trending Kepler and TESS

- get table of Kepler exoplanet system light curves
- fetch FLC
- get system info from table
- mask transits
- apply custom detrending
- search flares
- save results
"""


import copy
import time

from funcs.notebook import *
from funcs.detrend import estimate_detrended_noise, fit_spline, remove_sines_iteratively, remove_exponential_fringes
from funcs.transitmask import get_full_transit_mask

from altaipony.lcio import from_mast, from_path
from altaipony.flarelc import FlareLightCurve
from altaipony.altai import find_iterative_median

from lightkurve import search_lightcurvefile

from astropy.io import fits


def custom_detrending(lc, spline_coarseness=30, spline_order=3,
                      savgol1=6., savgol2=3., pad=3):
    """Custom de-trending for TESS and Kepler 
    short cadence light curves, including TESS Cycle 3 20s
    cadence.
    
    Parameters:
    ------------
    lc : FlareLightCurve
        light curve that has at least time, flux and flux_err
    spline_coarseness : float
        time scale in hours for spline points. 
        See fit_spline for details.
    spline_order: int
        Spline order for the coarse spline fit.
        Default is cubic spline.
    savgol1 : float
        Window size for first Savitzky-Golay filter application.
        Unit is hours, defaults to 6 hours.
    savgol2 : float
        Window size for second Savitzky-Golay filter application.
        Unit is hours, defaults to 3 hours.
    pad : 3
        Outliers in Savitzky-Golay filter are padded with this
        number of data points. Defaults to 3.
        
    Return:
    -------
    FlareLightCurve with detrended_flux attribute
    """
    dt = np.mean(np.diff(lc.time.value))
#     plt.figure(figsize=(16,8))
#     plt.xlim(20,21)
#     plt.plot(lc.time, lc.flux+2500, c="c", label="original light curve")
#     t0 = time.clock()
    # fit a spline to the general trends
    lc1, model = fit_spline(lc, spline_order=spline_order,
                            spline_coarseness=spline_coarseness)
    
    # replace for next step
    lc1.flux = lc1.detrended_flux.value
#     t1 = time.clock()
#     plt.plot(lc1.time, model+2500, c="r", label="rough trends")
#     plt.plot(lc1.time, lc1.detrended_flux+500, c="orange", label="rough trends removed")

    # removes strong and fast variability on 5 day to 4.8 hours 
    # simple sines are probably because rotational variability is 
    # either weak and transient or strong and persistent on the timescales
    lc2 = lc1#remove_sines_iteratively(lc1)
#     t2 = time.clock()
#     plt.plot(lc2time, lc2.detrended_flux-200, label="sines removed")
    
    # choose a 6 hour window
    w = int((np.rint(savgol1 / 24. / dt) // 2) * 2 + 1)

    # use Savitzy-Golay to iron out the rest
    lc3 = lc2.detrend("savgol", window_length=w, pad=pad)
#     t3 = time.clock()
    # choose a three hour window
    w = int((np.rint(savgol2 / 24. / dt) // 2) * 2 + 1)

    # use Savitzy-Golay to iron out the rest
    lc4 = lc3.detrend("savgol", window_length=w, pad=pad)
#     t4 = time.clock()
#     plt.plot(lc4.time, lc4.detrended_flux-800, c="k", label="SavGol applied")
    
    # find median value
    lc4 = find_iterative_median(lc4)
#     t41 = time.clock()
    # replace for next step
    lc4.flux = lc4.detrended_flux.value
    
    # remove exopential fringes that neither spline, 
    # nor sines, nor SavGol 
    # can remove.
    lc5 = remove_exponential_fringes(lc4)
#     t5 = time.clock()
#     plt.plot(lc5.time, lc5.detrended_flux, c="magenta", label="expfunc applied")
#     print(t1-t0, t2-t1, t3-t2, t4-t3, t41-t4, t5-t41, t5-t0)
#     plt.xlim(10,40)
#     plt.xlabel("time [days]")
#     plt.ylabel("flux")
#     plt.legend()    
    return lc5


def add_meta_data_and_write(ff, dflcn, ID, TIC, sector, mission,
                  lc_n, w, tstamp, mask_pos_outliers_sigma):
    """Write out flare table to file."""
    
    
    if ff.shape[0]==0:
        ff["phase"]=-1
        ff["total_n_valid_data_points"] = dflcn.detrended_flux.shape[0]
        ff["ID"] = ID
        ff["TIC"] = TIC
        ff["qcs"] = sector
        ff["mission"] = mission
        ff["tstamp"] = tstamp
        ff["lc_n"] = lc_n
        ff["w"] = w
        ff["mask_pos_outliers_sigma"] = mask_pos_outliers_sigma
        ff["real"]=-1
        ff = ff.append({"phase":-1,
                        "total_n_valid_data_points":dflcn.detrended_flux.shape[0],
                        "ID":ID,
                        "TIC":TIC,
                        "qcs" : sector,
                        "mission":mission,
                        "tstamp":tstamp,
                        "lc_n":lc_n,
                        "w":w,
                        "mask_pos_outliers_sigma":mask_pos_outliers_sigma,
                        "real":-1},
                         ignore_index=True)

    # otherwise add ID, QCS and mission
    else:
        ff["total_n_valid_data_points"] = dflcn.detrended_flux.shape[0]
        ff["ID"] = ID
        ff["TIC"] = TIC
        ff["qcs"] = sector
        ff["mission"] = mission
        ff["tstamp"] = tstamp
        ff["lc_n"] = lc_n
        ff["w"] = w
        ff["mask_pos_outliers_sigma"] = mask_pos_outliers_sigma

    # add results to file
    with open("../results/2022_07_flares.csv", "a") as file:
        ff.to_csv(file, index=False, header=False)
            
def write_flc_to_file(dflcn, flc, path_dflcn):
    """Write detrended light curve to fits."""
    
    
    dflcn.to_fits(path_dflcn, 
                  FLUX=flc.flux.value,
                  DETRENDED_FLUX=dflcn.detrended_flux.value,
                  DETRENDED_FLUX_ERR=dflcn.detrended_flux_err.value,
                  IT_MED=dflcn.it_med.value,
                  FLUX_MODEL=dflcn.flux_model.value,
                  PHASE = dflcn.phase,
                  overwrite=True)

def write_no_lc(input_target):
    with open("../results/2022_07_nolc.txt","a") as f:
        s = f"TIC {input_target.TIC}\n"
        f.write(s)
    
    
sep = "-----------------------------------------"

def mprint(message):
    print(sep)
    print(message)
    print(sep)
    
offset = {"K2":2454833.,
          "Kepler":2454833.,
          "TESS":2457000.,
          'Transiting Exoplanet Survey Satellite (TESS)':2457000.}    


Bad key legend.labelcolor in file /home/ekaterina/.config/matplotlib/matplotlibrc, line 530 ('legend.labelcolor:    None')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.4.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
def run_analysis(flc, input_target, sector, mission, lc_n, download_dir,
                 i=0, mask_pos_outliers_sigma = 2.5, addtail = True):
    # get timestamp for result
    tstamp = time.strftime("%Y_%m_%d", time.localtime())
    print(f"date: {tstamp}")

    dflc = custom_detrending(flc)
    print("LC successfully detrended.")

    # define two hour window for rolling std
    w = np.floor(1. / 12. / np.nanmin(np.diff(dflc.time.value)))
    if w%2==0: 
        w+=1

    # use window to estimate the noise in the LC
    dflcn = estimate_detrended_noise(dflc, std_window=int(w), 
                                  mask_pos_outliers_sigma=mask_pos_outliers_sigma)

    # search the residual for flares
    ff = dflcn.find_flares(addtail=addtail).flares


    # calculate the observed phases
    # calculate midtime of transit in TESS or Kepler time
    if mission == "TESS":
        if np.isfinite(input_target.pl_tranmid_tess):
            midtime = input_target.pl_tranmid_tess - offset[mission]
        else:
            midtime = input_target.pl_tranmid - offset[mission]
    elif mission == "Kepler":
        midtime = input_target.pl_tranmid - offset[mission]
    print(f"Transit midtime in {mission} time: {midtime}")

    # calculate phases for the light curve
    dflcn['phase'] = ((dflcn.time.value - midtime) % input_target.pl_orbper) / input_target.pl_orbper

    # calculate the phase at which the flare was observed
    ff["phase"] = ff.cstart.apply(lambda x: dflcn["phase"][np.where(x==dflcn.cadenceno)][0])
    

    # this is just to get the order of columns right, will be added later again
    if ff.shape[0]>0:
        del ff["total_n_valid_data_points"]

    # chop out all phases where we have no data points to look for flares in:
    dflcn["phase"][~np.isfinite(dflcn["detrended_flux"])] = np.nan

    fshow = ff[["tstart",'tstop',"phase","ampl_rec","dur"]]
    if fshow.shape[0]>0:
        print(f"Flares found:\n{fshow}")
    else:
        print(f'No flares found in LC.')

    # add meta info to flare table
    # if no flares found, add empty row and write to file
    add_meta_data_and_write(ff, dflcn, input_target.hostname, 
                            input_target.TIC, sector,
                            mission, lc_n, w, tstamp,
                            mask_pos_outliers_sigma)


    #write out detrended light curve
    if mission=="TESS":
        path_dflcn = f"{download_dir}/{tstamp}_{input_target.TIC}_{sector}_altai_{i}.fits"
    elif mission=="Kepler":
        name = input_target.hostname.replace(" ","_").replace("-","_")
        path_dflcn = f"{download_dir}/{tstamp}_{input_target.hostname}_{sector}_altai_{i}.fits"
        
    write_flc_to_file(dflcn, flc, path_dflcn)
    print(f"Wrote out LC to {path_dflcn}.")

#     %matplotlib inline
#     plt.figure(figsize=(16,5))
    # plt.plot(flc.time, flc.flux)
    # plt.plot(dflcn.time.value, dflcn.detrended_flux.value/,c="orange")
#     plt.plot(flc.time.value, flc.pdcsap_flux.value/np.median(flc.pdcsap_flux.value),c="r")
#     plt.scatter(dflcn.time.value, dflcn.phase.value/10+1,c="k",zorder=10,s=.3)
#     plt.title(f"{mission} Sector/Quarter {sector}, TIC {input_target.TIC}, {input_target.hostname}, {cadence} cadence.")
    # plt.plot(flc.time.value, flc.flux.value,c="k")
    # x=25
    # plt.xlim(2412,2414)
    # plt.ylim(0.99,1.3)
    return ff.shape[0]

def get_table_of_light_curves(input_target):

    try:
        lcs  = search_lightcurvefile(input_target.hostname)   
        conditions = (lcs.exptime.value < 130) & (lcs.author != "TASOC")
        lcs_sel = lcs[conditions]
    except KeyError:
        try:
            lcs  = search_lightcurvefile(f"TIC {input_target.TIC}")
            conditions = (lcs.exptime.value < 130) & (lcs.author != "TASOC")
            lcs_sel = lcs[conditions]
        except KeyError:
            write_no_lc(input_target)
            return
    if len(lcs)==0:
        write_no_lc(input_target)
        return
   
    
    lcs_sel = lcs_sel.table.to_pandas().sort_values(by="t_exptime")
    lcs_sel_tess = lcs_sel.loc[lcs_sel.mission.str[:4]=="TESS",:].drop_duplicates(subset=["mission"],keep="first")
    lcs_sel_kepler = lcs_sel[lcs_sel.mission.str[:6]=="Kepler"].drop_duplicates(subset=["mission"])

    lcs_sel = pd.concat([lcs_sel_kepler,lcs_sel_tess])
   
    return lcs_sel

In [3]:
# Composite Table of confirmed exoplanets
path = "../data/2022_07_27_input_catalog_star_planet_systems.csv"

mprint(f"[UP] Using compiled input catalog from {path}")

input_catalog = pd.read_csv(path) 

-----------------------------------------
[UP] Using compiled input catalog from ../data/2022_07_27_input_catalog_star_planet_systems.csv
-----------------------------------------


In [4]:
lcs_sel=pd.DataFrame()
lcs_sel.shape[0]

0

In [8]:
input_catalog[input_catalog.hostname == "Kepler-1313"]


,TIC,hostname,pl_name,sy_pnum,sy_snum,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_tranmid,pl_tranmiderr1,pl_tranmiderr2,pl_orbper_tess,pl_orbpererr1_tess,pl_orbpererr2_tess,pl_tranmid_tess,pl_tranmiderr1_tess,pl_tranmiderr2_tess,obrparams_tess
2829,63213622,Kepler-1313,Kepler-1313 b,1.0,1.0,3.833091,0.000005,-0.000005,2.454965e+06,0.00115,-0.00115,NaN,NaN,NaN,NaN,NaN,NaN,0


In [9]:
count = 2830
#Kepler-24 is the next one in line!

In [ ]:
Nflares = 0 
while Nflares < 1000:
    lcs_sel=pd.DataFrame()
    print(lcs_sel.shape)
    while lcs_sel.shape[0]==0:
        input_target = input_catalog.iloc[count]

        lcs_sel = get_table_of_light_curves(input_target)
        if lcs_sel is None:
            lcs_sel=pd.DataFrame()
        count+=1

    TIC = "TIC " + str(input_target.TIC)
    ID = input_target.hostname
    n = 0
    Nflares = 0
    while n<lcs_sel.shape[0]:

        sector = lcs_sel.iloc[n].mission[-2:]
        mission = lcs_sel.iloc[n].mission.split(" ")[0]

        lc_n = n + 1

        if lcs_sel.iloc[n].exptime < 30:
            cadence = "fast"
        else: 
            cadence = "short"

        print(f"Get {mission} Sector/Quarter {sector}, {TIC}, {ID}, {cadence} cadence.")

        # fetch light curve from MAST
        download_dir = "/home/ekaterina/Documents/001_science/lcs"

        if mission=="TESS":
            flc = from_mast(TIC, mission=mission, c=sector,
                        cadence=cadence, author="SPOC",
                        download_dir=download_dir)
            if flc is None:
                print(f"No LC found for {mission}, {ID}, Quarter {sector}.")
                with open("../results/2022_07_listed_but_nothing_found.txt", "a") as f:
                    string = f"{mission},{ID},{TIC},{sector},{cadence}\n"
                    f.write(string)
                n += 1
            else:
                Nflares += run_analysis(flc, input_target, sector, mission, lc_n, download_dir, i=0)
                n += 1
        elif mission=="Kepler":
            flcl = from_mast(ID, mission=mission, c=sector,
                        cadence=cadence,
                        download_dir=download_dir)

            if flcl is None:
                print(f"No LC found for {mission}, {ID}, Quarter {sector}.")
                with open("../results/2022_07_listed_but_nothing_found.txt", "a") as f:
                    string = f"{mission},{ID},{TIC},{sector},{cadence}\n"
                    f.write(string)
                n += 1

            elif type(flcl) != list:

                print(f"1 LC found for {mission}, {ID}, Quarter {sector}.")
                Nflares += run_analysis(flcl, input_target, sector, mission, lc_n, download_dir, i=0)
                n += 1



            else:
                print(f"{len(flcl)} LCs found for {mission}, {ID}, Quarter {sector}.")
                for i, flc in enumerate(flcl):
                    Nflares += run_analysis(flc, input_target, sector, mission, lc_n, download_dir, i=i)

                n += 1

    print(f"\n---------------------\n{Nflares} flares found!\n-------------------\n")
print(f"\nNext count is {count}.\n")

(0, 0)


/home/ekaterina/Documents/000_envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:88: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
/home/ekaterina/Documents/000_envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:88: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
/home/ekaterina/Documents/000_envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:88: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
Could not resolve Kepler-1918 to a sky position.
Could not resolve Kepler-1918 to a sky position.
/home/ekaterina/Documents/000_envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:93: LightkurveDeprecationWarning: 

Get Kepler Sector/Quarter 07, TIC 63283780, Kepler-210, short cadence.
No LC found for Kepler, Kepler-210, Quarter 07.
Get Kepler Sector/Quarter 17, TIC 63283780, Kepler-210, short cadence.


/home/ekaterina/Documents/000_envs/python37/lib/python3.7/site-packages/lightkurve/io/kepler.py:38: ResourceWarning: unclosed file <_io.FileIO name='/home/ekaterina/Documents/001_science/lcs/mastDownload/Kepler/kplr007447200_sc_Q000000033333323332/kplr007447200-2013121191144_slc.fits' mode='rb' closefd=True>
  time_format="bkjd",
1% (248/32850) of the cadences will be ignored due to the quality mask (quality_bitmask=1130799).
1% (248/32850) of the cadences will be ignored due to the quality mask (quality_bitmask=1130799).
/home/ekaterina/Documents/000_envs/python37/lib/python3.7/site-packages/lightkurve/io/kepler.py:38: ResourceWarning: unclosed file <_io.FileIO name='/home/ekaterina/Documents/001_science/lcs/mastDownload/Kepler/kplr007447200_sc_Q000000033333323332/kplr007447200-2013131215648_slc.fits' mode='rb' closefd=True>
  time_format="bkjd",
0% (28/6180) of the cadences will be ignored due to the quality mask (quality_bitmask=1130799).
0% (28/6180) of the cadences will be ignored

2 LCs found for Kepler, Kepler-210, Quarter 17.
date: 2022_08_02
LC successfully detrended.


...

date: 2022_08_02
LC successfully detrended.


/home/ekaterina/Documents/000_envs/python37/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
Found 0 candidate(s) in the (0,52371) gap.
Found 0 candidate(s) in the (52371,104771) gap.


Transit midtime in TESS time: 2253.248463000171
No flares found in LC.
Wrote out LC to /home/ekaterina/Documents/001_science/lcs/2022_08_02_98283926_33_altai_0.fits.
Get TESS Sector/Quarter 34, TIC 98283926, HATS-42, fast cadence.


/home/ekaterina/Documents/000_envs/python37/lib/python3.7/site-packages/lightkurve/io/tess.py:34: ResourceWarning: unclosed file <_io.FileIO name='/home/ekaterina/Documents/001_science/lcs/mastDownload/TESS/tess2021014023720-s0034-0000000098283926-0204-a_fast/tess2021014023720-s0034-0000000098283926-0204-a_fast-lc.fits' mode='rb' closefd=True>
  lc = read_generic_lightcurve(filename, flux_column=flux_column, time_format="btjd")
0% (210/104760) of the cadences will be ignored due to the quality mask (quality_bitmask=175).
0% (210/104760) of the cadences will be ignored due to the quality mask (quality_bitmask=175).


date: 2022_08_02


In [ ]:
type(flcl) != list

In [ ]:
# 2022_07_29_Kepler-974_14_altai_2
# 2022_07_29_Kepler-974_15_altai_2
# 2022_07_29_Kepler-974_16_altai_1.fits +2
fff = fits.open("/home/ekaterina/Documents/001_science/lcs/2022_07_29_Kepler-350_14_altai_2.fits")[1].data
ff = pd.read_csv("../results/2022_07_flares.csv")
ff = ff[(ff.ID == ID) & (ff.qcs==14)]
ff

In [ ]:
%matplotlib inline
for j, flare in ff.iloc[2:].iterrows():
    plt.figure(figsize=(16,5))
    cap=.5
    ts, tf = flare.tstart, flare.tstop
    print(ts,tf)
    _ = fff[np.where((fff['time']>=ts-.1/cap) & (fff['time']<=tf+.1/cap))]
    med = np.median(_['flux'])
    plt.plot(_['time'], _['flux']/med, c="k")
    
    _ = fff[np.where((fff['time']>=ts-1e-8) & (fff['time']<=tf+1e-8))]
    plt.scatter(_['time'], _['flux']/med, c="r")

    plt.scatter(ts, flare.phase/5 + 1)
    plt.plot(_['time'], _['phase']/5 + 1, c="grey")
#     plt.ylim(0.99,1.004)